In [3]:
user = User.objects.first()

In [4]:
invoice = user.stripe_customer.get_stripe_upcoming_invoice()
data = invoice.lines.data

In [7]:
data

[<InvoiceLineItem line_item id=ii_1EDkhFGjkoMLHlzNS0L8inET at 0x7fcad169ca98> JSON: {
   "amount": 259,
   "currency": "usd",
   "description": "Anorak Management Fee",
   "discountable": true,
   "id": "ii_1EDkhFGjkoMLHlzNS0L8inET",
   "invoice_item": "ii_1EDkhFGjkoMLHlzNS0L8inET",
   "livemode": false,
   "metadata": {},
   "object": "line_item",
   "period": {
     "end": 1552535853,
     "start": 1552535853
   },
   "plan": null,
   "proration": false,
   "quantity": 1,
   "subscription": "sub_Ee5GIgHDyjTmMk",
   "type": "invoiceitem"
 },
 <InvoiceLineItem line_item id=ii_1EDkdXGjkoMLHlzNMhqrLCM8 at 0x7fcad169cbf8> JSON: {
   "amount": 818,
   "currency": "usd",
   "description": "Remaining time on Netflix - Family Plan (4 users) after 14 Mar 2019",
   "discountable": false,
   "id": "ii_1EDkdXGjkoMLHlzNMhqrLCM8",
   "invoice_item": "ii_1EDkdXGjkoMLHlzNMhqrLCM8",
   "livemode": false,
   "metadata": {},
   "object": "line_item",
   "period": {
     "end": 1554102000,
     "start": 

In [16]:
for item in data:
    if(item.description == "Anorak Management Fee"):
        print(item)

{
  "amount": 259,
  "currency": "usd",
  "description": "Anorak Management Fee",
  "discountable": true,
  "id": "ii_1EDkhFGjkoMLHlzNS0L8inET",
  "invoice_item": "ii_1EDkhFGjkoMLHlzNS0L8inET",
  "livemode": false,
  "metadata": {},
  "object": "line_item",
  "period": {
    "end": 1552535853,
    "start": 1552535853
  },
  "plan": null,
  "proration": false,
  "quantity": 1,
  "subscription": "sub_Ee5GIgHDyjTmMk",
  "type": "invoiceitem"
}
